## Scope of Exercise 3

The goal of this project is to collect data about cities and countries, clean and merge the data to answer the following questions:

* How do the rankings change over time?
* How do these rankings correlate with each other for one specific year?
* How do they correlate with other statistics about the countries in which
  the cities are located?
* How do they correlate with quality of life rankings of the country
  in which the cities are located?
* What are the determining characteristics for livability of a city?
  How do they correlate with cost of living?

## Imports

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
import statsmodels.formula.api as smf

## Load Data

### Numbeo Cities

In [ ]:
# from https://www.numbeo.com/quality-of-life/rankings_by_country.jsp?title=2012-Q1
dfNumbeo = pd.read_csv(r"data/Numbeo.csv",sep=";",decimal=",")
dfNumbeo.columns = ['Period', 'City', 'NumbeoIndex', 'PurchasingPower', 'Safety', 'HealthCare', 'CostOfLiving', 'PropertyPriceToIncomeRatio', 'TrafficCommuteTime','Pollution', 'Climate']

dfNumbeo[['City','Country','Country2']] = dfNumbeo['City'].str.split(',',expand=True)
dfNumbeo['City'] = dfNumbeo.City.str.strip()
dfNumbeo['Country'] = dfNumbeo.Country.str.strip()
dfNumbeo.set_index('City',inplace=True, drop=False)
# fix of USA cities/country
dfNumbeo.loc['San Francisco','Country'] = "United States"
dfNumbeo.loc['Los Angeles','Country'] = "United States"
dfNumbeo.loc['New York','Country'] = "United States"

# why does it introduce another column?
# dfNumbeo.to_csv("numbeoprocessed.csv", index=False)
# Answer: United States is part of the city name, so dropping this column...
dfNumbeo.drop('Country2', axis=1, inplace=True)

dfNumbeo2012 = dfNumbeo.set_index('Period').loc['2012-Q1',:]
dfNumbeo2012.reset_index(inplace=True)
dfNumbeo2012['Country'] = dfNumbeo2012.Country.str.strip()
dfNumbeo2012.set_index('City',inplace=True, drop=False)
dfNumbeo2012["NumbeoRank"] = dfNumbeo2012["NumbeoIndex"].rank(ascending=False) 

### Mercer Cities

In [ ]:
##loading mercer data extracted from https://en.wikipedia.org/wiki/Mercer_Quality_of_Living_Survey
dfMercer = pd.read_csv('data/Mercer.csv')
dfMercer.columns = ['City','Country','MercerIndex','Year','MercerRank']

dfMercer2012 = dfMercer.loc[dfMercer['Year']== 2012,:]
dfMercer2012.set_index('City',inplace=True, drop=False)

### UN Prosperity Cities

In [ ]:
#Source: United Nations Human Settlements Programme (UN-Habitat), Global Urban Indicators Database 2012.
dfProsperity2012 = pd.read_csv('data/city-prosperity-index_country.csv')
dfProsperity2012.columns = ['Country','City','Cpi5','Cpi4','Productivity','QoLUN','Infrastructure','Environment','Equity']

dfProsperity2012 = dfProsperity2012.set_index('City', drop=False)
dfProsperity2012["UNRank"] = dfProsperity2012["QoLUN"].rank(ascending=False)

### Human Development Index (HDI) Countries

In [ ]:
# from http://hdr.undp.org/en/data#
hdi = pd.read_csv('./data/human-development-index.csv', header=1)

hdi["HDIRank(2012)"] = hdi["2012"].rank(method='dense', ascending=False)

# convert countries to numbeo notation
hdi.loc[hdi['Country'] == 'Czechia', 'Country'] = 'Czech Republic'
hdi.loc[hdi['Country'] == 'Iran (Islamic Republic of)', 'Country'] = 'Iran'
hdi.loc[hdi['Country'] == 'Russian Federation', 'Country'] = 'Russia'
hdi.loc[hdi['Country'] == 'Bolivia (Plurinational State of)', 'Country'] = 'Bolivia'
hdi.loc[hdi['Country'] == 'Moldova (Republic of)', 'Country'] = 'Moldova'
hdi.loc[hdi['Country'] == 'Korea (Republic of)', 'Country'] = 'Republic of Korea'
hdi.loc[hdi['Country'] == 'Tanzania (United Republic of)', 'Country'] = 'United Republic of Tanzania'

### Numbeo Countries

In [ ]:
dfNumbeo2012Countries = pd.read_csv(r"./data/numbeo-by-country.csv", header=0)
dfNumbeo2012Countries["NumbeoCountryRank"] = dfNumbeo2012Countries["Rank"]

### Gapminder Datasets

#### Life Expectancy
The average number of years a newborn child would live if current mortality patterns were to stay the same
[Source](http://gapm.io/ilex)

#### CO2 Emissions
Carbon dioxide emissions from the burning of fossil fuels (mteric tonnes of CO2 per person). [Source](https://cdiac.ess-dive.lbl.gov/), [Source](http://gapm.io/ilex)

#### Gross domestic income per person
Gross domestic product per person adjusted for differences in purchasing power (in international dollars, fixed 2011 prices, PPP based on 2011 ICP). [Source](http://gapm.io/dgdppc)

#### Children per woman
Total fertility rate. The number of children that would be born to each women with prevailing age-specific fertility rates [Source](http://gapm.io/dtfr)

#### Number of child deaths
The number of children dying before age 5. [Source](https://www.who.int/healthinfo/global_burden_disease/en/)

#### Population total
Total population of the country [Source](http://gapm.io/dpop)

#### Unemployment rates
- Age 15+: Percentage of total polulation, age group above 15, that has been registered as unemployed during the given year [Source](https://www.ilo.org/ilostat/)
- Age 15-24: Percentage of total polulation, age group 15-24, that has been registered as unemployed during the given year [Source](https://www.ilo.org/ilostat/)
- Age 25-54: Percentage of total polulation, age group 25-54, that has been registered as unemployed during the given year [Source](https://www.ilo.org/ilostat/)
- Age 55-64: Percentage of total polulation, age group 55-64, that has been registered as unemployed during the given year [Source](https://www.ilo.org/ilostat/)
- Age 64+: Percentage of total polulation, age group above 65, that has been registered as unemployed during the given year [Source](https://www.ilo.org/ilostat/)

In [ ]:
dfGapminder = pd.read_csv(r"data/gapminder_life_expectancy_years.csv",sep=",",decimal=".")
dfLifeExpectancyYears = dfGapminder.iloc[:,[0,-7]]
dfLifeExpectancyYears = dfLifeExpectancyYears.rename(columns={'2012': 'life_expectancy_[years]'})

dfGapminder = pd.read_csv(r"data/gapminder_co2_emissions_tonnes_per_person.csv",sep=",",decimal=".")
dfCo2Emissions = dfGapminder.iloc[:,[0,-3]]
dfCo2Emissions = dfCo2Emissions.rename(columns={'2012': 'co2_emissions_year_[tons]'})

dfGapminder = pd.read_csv(r"data/gapminder_income_per_person_gdppercapita_ppp_inflation_adjusted.csv",sep=",",decimal=".")
dfIncome = dfGapminder.iloc[:,[0,-29]]
dfIncome = dfIncome.rename(columns={'2012': 'income_per_person'})

dfGapminder = pd.read_csv(r"data/gapminder_children_per_woman_total_fertility.csv",sep=",",decimal=".")
dfFertility = dfGapminder.iloc[:,[0,-7]]
dfFertility = dfFertility.rename(columns={'2012': 'fertility_children_per_woman'})

dfGapminder = pd.read_csv(r"data/gapminder_number_of_child_deaths.csv",sep=",",decimal=".")
dfChildDeaths = dfGapminder.iloc[:,[0,-4]]
dfChildDeaths = dfChildDeaths.rename(columns={'2012': 'child_deaths'})

dfGapminder = pd.read_csv(r"data/gapminder_population_total.csv",sep=",",decimal=".")
dfPopulation = dfGapminder.iloc[:,[0,-89]]
dfPopulation = dfPopulation.rename(columns={'2012': 'population_total'})

dfGapminder = pd.read_csv(r"data/gapminder_aged_15plus_unemployment_rate_percent.csv",sep=",",decimal=".")
filled = dfGapminder.iloc[:,1:].fillna(method="ffill", axis=1).fillna(method="bfill", axis=1)
filled['country'] = dfGapminder.iloc[:,0]
dfUnemployment15plus = filled.iloc[:,[-1, -7]]
dfUnemployment15plus = dfUnemployment15plus.rename(columns={'2012': 'unemployment_15plus'})

dfGapminder = pd.read_csv(r"data/gapminder_aged_15_24_unemployment_rate_percent.csv",sep=",",decimal=".")
filled = dfGapminder.iloc[:,1:].fillna(method="ffill", axis=1).fillna(method="bfill", axis=1)
filled['country'] = dfGapminder.iloc[:,0]
dfUnemployment15_24 = filled.iloc[:,[-1, -7]]
dfUnemployment15_24 = dfUnemployment15_24.rename(columns={'2012': 'unemployment_15_24'})

dfGapminder = pd.read_csv(r"data/gapminder_aged_25_54_unemployment_rate_percent.csv",sep=",",decimal=".")
filled = dfGapminder.iloc[:,1:].fillna(method="ffill", axis=1).fillna(method="bfill", axis=1)
filled['country'] = dfGapminder.iloc[:,0]
dfUnemployment25_54 = filled.iloc[:,[-1, -7]]
dfUnemployment25_54 = dfUnemployment25_54.rename(columns={'2012': 'unemployment_25_54'})

dfGapminder = pd.read_csv(r"data/gapminder_aged_55_64_unemployment_rate_percent.csv",sep=",",decimal=".")
filled = dfGapminder.iloc[:,1:].fillna(method="ffill", axis=1).fillna(method="bfill", axis=1)
filled['country'] = dfGapminder.iloc[:,0]
dfUnemployment55_64 = filled.iloc[:,[-1, -7]]
dfUnemployment55_64 = dfUnemployment55_64.rename(columns={'2012': 'unemployment_55_64'})

dfGapminder = pd.read_csv(r"data/gapminder_aged_65plus_unemployment_rate_percent.csv",sep=",",decimal=".")
filled = dfGapminder.iloc[:,1:].fillna(method="ffill", axis=1).fillna(method="bfill", axis=1)
filled['country'] = dfGapminder.iloc[:,0]
dfUnemployment65plus = filled.iloc[:,[-1, -7]]
dfUnemployment65plus = dfUnemployment65plus.rename(columns={'2012': 'unemployment_65plus'})


In [ ]:
# merge dataset
dfGapminder = dfPopulation.merge(dfChildDeaths, how='left', left_on='country', right_on='country') 
dfGapminder = dfGapminder.merge(dfIncome, how='left', left_on='country', right_on='country') 
dfGapminder = dfGapminder.merge(dfCo2Emissions, how='left', left_on='country', right_on='country') 
dfGapminder = dfGapminder.merge(dfLifeExpectancyYears, how='left', left_on='country', right_on='country') 
dfGapminder = dfGapminder.merge(dfFertility, how='left', left_on='country', right_on='country') 
dfGapminder = dfGapminder.merge(dfUnemployment15plus, how='left', left_on='country', right_on='country') 
dfGapminder = dfGapminder.merge(dfUnemployment15_24, how='left', left_on='country', right_on='country') 
dfGapminder = dfGapminder.merge(dfUnemployment25_54, how='left', left_on='country', right_on='country') 
dfGapminder = dfGapminder.merge(dfUnemployment55_64, how='left', left_on='country', right_on='country') 
dfGapminder = dfGapminder.merge(dfUnemployment65plus, how='left', left_on='country', right_on='country')

dfGapminder = dfGapminder.dropna()
# print(dfGapminder.shape)
dfGapminder.loc[:, 'country'] = dfGapminder['country'].str.strip()

In [ ]:
# create ranks
dfGapminder.loc[:, 'rank_population'] = dfGapminder['population_total'].rank(ascending=False)
dfGapminder.loc[:, 'rank_child_deaths'] = dfGapminder['child_deaths'].rank(ascending=False)
dfGapminder.loc[:, 'rank_income_per_person'] = dfGapminder['income_per_person'].rank(ascending=False)
dfGapminder.loc[:, 'rank_co2_emissions'] = dfGapminder['co2_emissions_year_[tons]'].rank(ascending=False)
dfGapminder.loc[:, 'rank_life_expectancy'] = dfGapminder['life_expectancy_[years]'].rank(ascending=False)
dfGapminder.loc[:, 'rank_fertility'] = dfGapminder['fertility_children_per_woman'].rank(ascending=False)
dfGapminder.loc[:, 'rank_unemployment_15plus'] = dfGapminder['unemployment_15plus'].rank(ascending=False)
dfGapminder.loc[:, 'rank_unemployment_15_24'] = dfGapminder['unemployment_15_24'].rank(ascending=False)
dfGapminder.loc[:, 'rank_unemployment_25_54'] = dfGapminder['unemployment_25_54'].rank(ascending=False)
dfGapminder.loc[:, 'rank_unemployment_55_64'] = dfGapminder['unemployment_55_64'].rank(ascending=False)
dfGapminder.loc[:, 'rank_unemployment_65plus'] = dfGapminder['unemployment_65plus'].rank(ascending=False)

### WHO Data Sets

These datasets are available per country.
To identify a country the _ISO 3166-1 alpha-3 codes_ are used (field `COUNTRY`).

#### Sanitation

This dataset provides the usage of safely managed sanitation services
in percent of the population.
It is separated between the percentage in urban areas and in rural areas,
where we have used the urban-related ratio only.

#### Health Regulations Capacity

This dataset provides the average of 13 International Health Regulations
core capacity scores, based on the Sustainable Development Goals of the UN.

#### Helper Table: ISO Codes

To merge the tables with the rankings we added a table with country ISO codes.

In [ ]:
# Read in WHO data: Usage of safely managed sanitation services
dfWHO_sanitation = pd.read_csv('data/WHO_WSH_SANITATION_SAFELY_MANAGED.csv')
# Read in WHO data: sustainable development goals
dfWHO_SDG = pd.read_csv('data/WHO_SDGIHR.csv')
# Read in table of ISO codes
dfISO = pd.read_csv('data/country-iso.csv')
dfISO = dfISO.set_index('alpha-3')
# Enrich WHO data with country names
dfWHO_sanitation = dfWHO_sanitation.join(
    dfISO['name'], on='COUNTRY', how='inner')
dfWHO_SDG = dfWHO_SDG.join(
    dfISO['name'], on='COUNTRY', how='inner')
# filter for urban area (sanitation only)
# only use columns year and numeric
dfWHO_sanitation = dfWHO_sanitation.loc[
    dfWHO_sanitation.RESIDENCEAREATYPE == 'URB',
    ['name', 'YEAR', 'Numeric']]
dfWHO_SDG = dfWHO_SDG.loc[:, ['name', 'YEAR', 'Numeric']]

### WorldBank Data

#### GDP for all years - purchase-power adjusted

This data is similar to the GDP data downloaded via Gap Minder.
This time however the prices are not adjusted for the 2011 value but
only adjusted for purchase power.

#### Relative Costs of Emissions

This data is created by WorldBank to estimate the costs of emissions
in the countries, relative to the net income of the population.

In [ ]:
# Read in GDP data from WorldBank
dfWorldbankGDP = pd.read_csv(
    'data/Worldbank_GDP_per_capita.csv',
    skiprows=4)
# make Country Name into index
dfWorldbankGDP = dfWorldbankGDP.set_index('Country Name')
# Read in Emissions costs data from WorldBank
dfWorldbankEmissions = pd.read_csv(
    'data/Worldbank_Emissions_Damage.csv',
    skiprows=4)
# make Country Name into index
dfWorldbankEmissions = dfWorldbankEmissions.set_index('Country Name')

In [ ]:
def print_heatmap(df, title):
    fig = plt.figure(figsize=(15,15))
    fig.suptitle(title, fontsize='x-large', fontweight='bold', y=0.93)
    g = sns.heatmap(df.corr(), vmin=-1, cmap='coolwarm', annot=True);
    plt.xlabel('', fontsize=14)
    plt.ylabel('', fontsize=14);
    g.set_xticklabels(g.get_xticklabels(), rotation=30, horizontalalignment='right')
    file_name = title.replace(' ', '_')
    fig.savefig(f'visuals/heatmap_{file_name}.png')
    
def print_scattermatrix(df, title):
    fig = plt.figure(figsize=(15,15))
    fig.suptitle(title, fontsize='x-large', fontweight='bold', y=0.93)
    g = sns.pairplot(df, kind="reg")
    plt.xlabel('', fontsize=14)
    plt.ylabel('', fontsize=14);
    file_name = title.replace(' ', '_')
    fig.savefig(f'visuals/scattermatrix_{file_name}.png')

Correlations with Mercer

In [ ]:
rank_columns = ['country', 'rank_population', 'rank_child_deaths', 'rank_income_per_person', 'rank_co2_emissions', 'rank_life_expectancy', 'rank_fertility', 'rank_unemployment_15plus', 'rank_unemployment_15_24', 'rank_unemployment_25_54', 'rank_unemployment_55_64', 'rank_unemployment_65plus']
dfMergedMercer = dfMercer2012.merge(dfGapminder[rank_columns], how='left', left_on='Country', right_on='country') 
# print(dfMergedMercer.shape)
dfMergedMercer.set_index('City',inplace=True)
# dfMergedMercer

Print correlations of rankings

In [ ]:
# population, gdp, co2, child_deaths, life_expectency, fertility stats
rank_columns = ['MercerRank', 'rank_population', 'rank_child_deaths', 'rank_income_per_person', 'rank_co2_emissions', 'rank_life_expectancy', 'rank_fertility']
print_scattermatrix(dfMergedMercer.loc[:,rank_columns], "Correlation of Mercer City and Gapminder Rankings")
print_heatmap(dfMergedMercer.loc[:,rank_columns], 'Correlation between Mercer City rank and Gapminder Rankings')
# unemployment stats
rank_columns = ['MercerRank', 'rank_unemployment_15plus', 'rank_unemployment_15_24', 'rank_unemployment_25_54', 'rank_unemployment_55_64', 'rank_unemployment_65plus']
print_scattermatrix(dfMergedMercer.loc[:,rank_columns], 'Correlation between Mercer City rank and Gapminder Unemployment Rankings')
print_heatmap(dfMergedMercer.loc[:,rank_columns], 'Correlation between Mercer City rank and Gapminder Unemployment Rankings')

Correlations with Numbeo

In [ ]:
rank_columns = ['country', 'rank_population', 'rank_child_deaths', 'rank_income_per_person', 'rank_co2_emissions', 'rank_life_expectancy', 'rank_fertility', 'rank_unemployment_15plus', 'rank_unemployment_15_24', 'rank_unemployment_25_54', 'rank_unemployment_55_64', 'rank_unemployment_65plus']
dfMergedNumbeo = dfNumbeo2012.merge(dfGapminder[rank_columns], how='left', left_on='Country', right_on='country') 
# print(dfMergedNumbeo.shape)
dfMergedNumbeo.set_index('City',inplace=True)
# dfMergedNumbeo = dfMergedNumbeo.dropna()

Print correlations with Numbeo

In [ ]:
# population, gdp, co2, child_deaths, life_expectency, fertility stats
rank_columns = ['NumbeoRank', 'rank_population', 'rank_child_deaths', 'rank_income_per_person', 'rank_co2_emissions', 'rank_life_expectancy', 'rank_fertility']
print_scattermatrix(dfMergedNumbeo.loc[:,rank_columns], 'Correlation of Numbeo and Gapminder Rankings')
print_heatmap(dfMergedNumbeo.loc[:,rank_columns], 'Correlation of Numbeo and Gapminder Rankings')
# unemployment stats
rank_columns = ['NumbeoRank', 'rank_unemployment_15plus', 'rank_unemployment_15_24', 'rank_unemployment_25_54', 'rank_unemployment_55_64', 'rank_unemployment_65plus']
print_scattermatrix(dfMergedNumbeo.loc[:,rank_columns], 'Correlation of Numbeo and Gapminder Unemployment Rankings')

Correlations with UN Prosperity

In [ ]:
rank_columns = ['country', 'rank_population', 'rank_child_deaths', 'rank_income_per_person', 'rank_co2_emissions', 'rank_life_expectancy', 'rank_fertility', 'rank_unemployment_15plus', 'rank_unemployment_15_24', 'rank_unemployment_25_54', 'rank_unemployment_55_64', 'rank_unemployment_65plus']
dfMergedProsperity = dfProsperity2012.merge(dfGapminder[rank_columns], how='left', left_on='Country', right_on='country') 
# print(dfMergedProsperity.shape)
dfMergedProsperity.set_index('City',inplace=True)
dfMergedProsperity = dfMergedProsperity.dropna()

Print correlations with UN Prosperity

In [ ]:
rank_columns = ['UNRank', 'rank_population', 'rank_child_deaths', 'rank_income_per_person', 'rank_co2_emissions', 'rank_life_expectancy', 'rank_fertility']
print_scattermatrix(dfMergedProsperity.loc[:,rank_columns], 'Correlation of UN Prosperity and Gapminder Rankings')
print_heatmap(dfMergedProsperity.loc[:,rank_columns], 'Correlation of UN Prosperity and Gapminder Rankings')
# unemployment stats
rank_columns = ['UNRank', 'rank_unemployment_15plus', 'rank_unemployment_15_24', 'rank_unemployment_25_54', 'rank_unemployment_55_64', 'rank_unemployment_65plus']
print_scattermatrix(dfMergedProsperity.loc[:,rank_columns], 'Correlation of UN Prosperity and Gapminder Unemployment Rankings')
print_heatmap(dfMergedProsperity.loc[:,rank_columns], 'Correlation of UN Prosperity and Gapminder Unemployment Rankings')

## How do the rankings change over time?
As a first step we look at the Numbeo Dataset and analyze the different indices visually over time

In [ ]:
display(dfNumbeo)

## What do these numbers mean? 
The Numbeos Quality of Life Ranking is a weighted index based on several key factors: 

### Purchasing Power
Relative to New York City: If domestic purchasing power is 40, this means that the inhabitants of that city with the average salary can afford to buy on an average 60% less goods and services than New York City residents with an average salary. 

### PropertyPriceToIncomeRatio
Generally calculated as the ratio of median apartment prices to median familial disposable income

### Cost of Living
These indices are relative to New York City (NYC). Which means that for New York City, each index should be 100(%). If another city has, for example, rent index of 120, it means that on an average in that city rents are 20% more expensive than in New York City.

### Pollution, Crime, Health Care
Is based on survey data of visitors. [-2, +2], with -2 having meaning of strongly negative and +2 meaning of strongly positive 

### Climate Index 
Is an estimation of the climate likability of a given city or a country. It is in the range [-100, +100] (higher is better)

### TrafficCommuteTime
Is a composite index of time consumed in traffic due to job commute, estimation of time consumption dissatisfaction, CO2 consumption estimation in traffic and overall inefficiencies in the traffic system

```
 protected void calculateIndex() {
    index = new TrafficIndex();
    index.time = overall.getTimeOverall();
    double tooMuchTime = 0.0;
    if (index.time > 25.0) {
      tooMuchTime = index.time - 25;
    }
    index.timeExp = index.time + Math.pow(tooMuchTime, Math.E);
    double co2 = 0.0;
    co2 += overall.time_bus * 20.0; // bus produces 20g of CO2 per minute (for each passenger)
    co2 += overall.time_driving * 133.0; // car produces 133g of CO2 per minute (assumes only driver)
    co2 += overall.time_train * 10.0; // train produces 10g of CO2 per minute (for each passenger)
    co2 += overall.time_tram * 15.0; // tram produces 15g of CO2 per minute (for each passenger)
    co2 += overall.time_other * 10.0; // other produces 10g of CO2 per minute
    co2 += overall.time_motorbike * 80.0; // motorbike produces 80g of CO2 per hour
    index.co2 = 2 * co2; 

    index.main = index.time + Math.sqrt(index.timeExp) + Math.sqrt(index.co2) + Math.sqrt(index.inefficiency);
}
```

### Quality of Life (QoL)
Note that the index is given by the following formular: 
```
QoL = Math.max(0, 100 + 
     purchasingPowerInclRentIndex / 2.5 -
     (housePriceToIncomeRatio * 1.0) -
     costOfLivingIndex / 10 +
     safetyIndex / 2.0 + healthIndex / 2.5 -
     trafficTimeIndex / 2.0 - pollutionIndex * 2.0 / 3.0 +
     climateIndex / 3.0); 
```

The formula has changed over time.
For example between June 2017 and Decembar 2017 they decreased the weight for the cost of living. 
_This should be considered when drawing conclusions later!_
Further details can be found on this website: https://www.numbeo.com/quality-of-life/indices_explained.jsp





In [ ]:
#How do the rankings change over time?

fig = plt.figure(figsize=(20,10))
fig.suptitle('Average Rankings over Time', fontsize='x-large', fontweight='bold')
#This is used for the main title. 'figure()' is a class that provides all the plotting elements of a diagram. 
#This must be used first or else the title will not show.fig.subplots_adjust(top=0.85) solves our overlapping title problem.
fig.subplots_adjust(top=0.95)


ax = fig.add_subplot(111)

fontdict={'fontsize': 14, 'fontweight' : 'book',    'verticalalignment': 'baseline',   'horizontalalignment': 'center'}

#ax.set_title('Plotting Tutorial', fontdict=fontdict)

sns.lineplot(x="Period", y="NumbeoIndex", data=dfNumbeo,legend='brief',label='Quality of Life')
sns.lineplot(x="Period", y="Climate", data=dfNumbeo,legend='brief',label='Climate')
sns.lineplot(x="Period", y="PurchasingPower", data=dfNumbeo,legend='brief',label='Purchasing Power')
sns.lineplot(x="Period", y="Safety", data=dfNumbeo,legend='brief',label='Safety')
sns.lineplot(x="Period", y="HealthCare", data=dfNumbeo,legend='brief',label='Health Care')
sns.lineplot(x="Period", y="CostOfLiving", data=dfNumbeo,legend='brief',label='Cost of Living')
sns.lineplot(x="Period", y="PropertyPriceToIncomeRatio", data=dfNumbeo,legend='brief',label='Property Price to Income Ratio')
sns.lineplot(x="Period", y="TrafficCommuteTime", data=dfNumbeo,legend='brief',label='Purchasing Power')
sns.lineplot(x="Period", y="Pollution", data=dfNumbeo,legend='brief',label='Pollution')

plt.legend(fontsize=14)
#plt.title('Average Rankings over Time', fontdict=fontdict, color="black", position=(0.5,1));
plt.xlabel('Collectiontime', fontsize=14)
plt.ylabel('Average Score', fontsize=14);
fig.savefig('visuals/RankingsOverTime.png')

In [ ]:
#There might be some outliers in the data so we decided to also look at a boxplot:
fig = plt.figure(figsize=(30,15))
fig.suptitle('Average Rankings over Time', fontsize='x-large', fontweight='bold')
#This is used for the main title. 'figure()' is a class that provides all the plotting elements of a diagram. 
#This must be used first or else the title will not show.fig.subplots_adjust(top=0.85) solves our overlapping title problem.
fig.subplots_adjust(top=0.95)


ax = fig.add_subplot(111)

fontdict={'fontsize': 14, 'fontweight' : 'book',    'verticalalignment': 'baseline',   'horizontalalignment': 'center'}

#ax.set_title('Plotting Tutorial', fontdict=fontdict)

dfNumbeoPivoted = pd.melt(dfNumbeo, col_level=0, id_vars=['Period', 'City'], value_vars=['NumbeoIndex', 'PurchasingPower', 'Safety', 'HealthCare', 'CostOfLiving', 'PropertyPriceToIncomeRatio', 'TrafficCommuteTime','Pollution', 'Climate'])


flierprops = dict(markerfacecolor='0.75', markersize=5,    linestyle='none')
g = sns.boxplot(x="Period", y="value",hue="variable",notch=True, data=dfNumbeoPivoted,order=np.flip(dfNumbeo.Period.unique()),flierprops=flierprops)

plt.legend(fontsize=14)
#plt.title('Average Rankings over Time', fontdict=fontdict, color="black", position=(0.5,1));
plt.xlabel('Collectiontime', fontsize=14)
plt.ylabel('Score', fontsize=14);
fig.savefig('visuals/RankingsOverTimeBoxPlot.png')

## Interpretation
The Quality of life rankings on average increased from 2012 to 2015. Then they decreased a bit till mid 2017 and recovered again to mid 2018. 
After that the quality of life ranking decreased again.


## How do city ratings correlate with quality of life rankings of the country in which the cities are located?


In [ ]:
print(f'number numbeo rows {dfNumbeo2012.shape[0]}')
numbeo2012_vs_hdi = pd.merge(dfNumbeo2012, hdi, how='inner', on='Country')
print(f'joined {numbeo2012_vs_hdi.shape[0]} cities from numbeo with countries from hdi')

**Common Correlation Function**

In [ ]:
def eval_corr(df, rankA, rankB, title):
    fig = plt.figure(figsize=(13,13))
    g = sns.pairplot(df.loc[:,[rankA,rankB]],kind="reg", diag_kind=None)
    g.fig.suptitle(title, y=0.97,fontsize=14)
    g.fig.subplots_adjust(top=0.85,left=0.2,bottom=0.1)
    g.fig.savefig(f'visuals/CorrelationOf{rankA}vs{rankB}.png')

    spearman_df = spearmanr(df[rankA], df[rankB])
    print(f'spearman relation between {rankA} and {rankB}: {spearman_df}')
    
    df["CityPerformance"] = df[rankA] - df[rankB]
    print(f'10 Cities which perform better than their country ({rankA} vs {rankB})')
    display(df.sort_values("CityPerformance", ascending=True)[["City", "Country", rankA, rankB]].head(10))
    print(f'10 Cities which perform worse than their country ({rankA} vs {rankB})')
    display(df.sort_values("CityPerformance", ascending=False)[["City", "Country", rankA, rankB]].head(10))

### Numbeo Cities vs Human Development Index (HDI) Countries

In [ ]:
eval_corr(numbeo2012_vs_hdi, 'NumbeoRank', 'HDIRank(2012)', "Numbeo Cities vs HDI Countries Rankings 2012")

### Numbeo Cities vs Human Development Index (HDI) Countries (reranked)

In [ ]:
numbeo2012_vs_hdi["HDIReRank(2012)"] = numbeo2012_vs_hdi["HDIRank(2012)"].rank(method='dense', ascending=True) 
eval_corr(numbeo2012_vs_hdi, 'NumbeoRank', 'HDIReRank(2012)', "Numbeo Cities vs HDI Countries Rankings Reranked 2012")

### Numbeo Cities vs Numbeo Countries

In [ ]:
numbeo2012_vs_numbeo2012_countries = pd.merge(dfNumbeo2012, dfNumbeo2012Countries, how='inner', on='Country')
print(f'joined {numbeo2012_vs_numbeo2012_countries.shape[0]} numbeo cities with countries')
eval_corr(numbeo2012_vs_numbeo2012_countries, 'NumbeoRank', 'NumbeoCountryRank', "Numbeo Cities vs Numbeo Countries Rankings 2012")

### Mercer Cities vs Human Development Index (HDI) Countries

In [ ]:
mercer2012_vs_hdi = pd.merge(dfMercer2012, hdi, how='inner', on='Country')
print(f'joined {mercer2012_vs_hdi.shape[0]} cities from mercer with countries from hdi')
eval_corr(mercer2012_vs_hdi, 'MercerRank', 'HDIRank(2012)', "Mercer Cities vs HDI Countries Rankings 2012")

### Mercer Cities vs Numbeo Countries

In [ ]:
mercer2012_vs_numbeo_countries = pd.merge(dfMercer2012, dfNumbeo2012Countries, how='inner', on='Country')
print(f'joined {mercer2012_vs_numbeo_countries.shape[0]} cities from mercer with countries from hdi')
eval_corr(mercer2012_vs_numbeo_countries, 'MercerRank', 'NumbeoCountryRank', "Mercer Cities vs Numbeo Countries Rankings 2012")

### UN Prosperity Cities vs HDI Countries

In [ ]:
prosperity2012_vs_hdi = pd.merge(dfProsperity2012, hdi, how='inner', on='Country')
print(f'joined {prosperity2012_vs_hdi.shape[0]} cities from prosperity with countries from hdi')
eval_corr(prosperity2012_vs_hdi, 'UNRank', 'HDIRank(2012)', "UN Prosperity Cities vs HDI Countries 2012")

### UN Prosperity Cities vs Numbeo Countries

In [ ]:
prosperity2012_vs_hdi = pd.merge(dfProsperity2012, dfNumbeo2012Countries, how='inner', on='Country')
print(f'joined {prosperity2012_vs_hdi.shape[0]} cities from prosperity with countries from numbeo')
eval_corr(prosperity2012_vs_hdi, 'UNRank', 'NumbeoCountryRank', "UN Prosperity Cities vs Numbeo Countries 2012")


## How do these rankings correlate with each other for one specific year?

In [ ]:
mergedDf = dfMercer2012.merge(dfNumbeo2012, left_index=True, right_index=True)

#now that we operate on a subset of the data we should rerank the cities to have comparable ranks! 
#otherwise we would have higher ranks for numbeo cities because we have more data there and our goal is to see if numbeo would sort the cities in the same order as mercer does. 
mergedDf["NumbeoReRank"] = mergedDf["NumbeoRank"].rank(ascending=False) 
mergedDf["MercerReRank"] = mergedDf["MercerRank"].rank(ascending=False) 
mergedDf.to_csv("data/NumbeoAndMercer.csv", index=False)
mergedDf

In [ ]:
fig = plt.figure(figsize=(13,13))
g = sns.pairplot(mergedDf.loc[:,['MercerReRank','NumbeoReRank']], kind="reg", diag_kind=None)
g.fig.suptitle("Correlation of Mercer and Numbeo Rankings \n with reranking in 2012", y=0.97,fontsize=14)
g.fig.subplots_adjust(top=0.85, left=0.2, bottom=0.1)
g.fig.savefig('visuals/CorrelationOfMercerAndNumbeoRanking2012withreranking.png')



In [ ]:
fig = plt.figure(figsize=(10,10))
g = sns.pairplot(mergedDf.loc[:,['MercerRank','NumbeoRank']], kind="reg", diag_kind=None)
g.fig.suptitle("Correlation of Mercer and Numbeo Rankings \n without reranking in 2012", y=0.97,fontsize=14)
g.fig.subplots_adjust(top=0.85,left=0.2,bottom=0.1)
g.fig.savefig('visuals/CorrelationOfMercerAndNumbeoRanking2012withoutreranking.png')

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R -o output
d <- read.csv2("data/NumbeoAndMercer.csv",sep=',',dec ='.')
rankcorrelation <- cor(d$NumbeoReRank, d$MercerReRank,  method = "spearman", use = "complete.obs")
indexcorrelation <- cor(d$NumbeoIndex, d$MercerIndex,  method = "spearman", use = "complete.obs")
output <- capture.output({c(rankcorrelation,indexcorrelation) })

In [ ]:
print(output)

## Insight
Interesting to see that Mercer Rankings correlate with Numbeo Rankings. Especially because Mercers sells the ranking data for 20k and Numbeo is (almost) for free. 

In [ ]:
mergedDf = mergedDf.merge(dfProsperity2012, left_index=True, right_index=True) 
mergedDf["UNReRank"] = mergedDf["UNRank"].rank(ascending=False) 
mergedDf

In [ ]:
fig = plt.figure(figsize=(10,10))
g = sns.pairplot(mergedDf.loc[:,['MercerReRank','NumbeoReRank','UNReRank']], kind="reg", diag_kind=None)
g.fig.suptitle("Correlation of Mercer, UN and Numbeo Rankings in 2012", y=0.97,fontsize=14)
g.fig.subplots_adjust(top=0.9)
g.fig.savefig('visuals/CorrelationOfMercerAndUNAndNumbeoRanking2012.png')


In [ ]:
#Todo merging only Numbeo with UN Data! (Removing the bottleneck Mercer)
mergedDf = dfProsperity2012.merge(dfNumbeo2012, left_index=True, right_index=True)
mergedDf["NumbeoReRank"] = mergedDf["NumbeoRank"].rank(ascending=False) 
mergedDf["UNReRank"] = mergedDf["UNRank"].rank(ascending=False) 
mergedDf

In [ ]:
fig = plt.figure(figsize=(10,10))

g = sns.pairplot(mergedDf.loc[:,['NumbeoReRank','UNReRank']], kind="reg", diag_kind=None)
g.fig.suptitle("Correlation of UN and Numbeo Rankings in 2012", y=0.97,fontsize=14)
g.fig.subplots_adjust(top=0.9)
g.fig.savefig('visuals/CorrelationOfUNAndNumbeoRanking2012.png')

## How do they correlate with other statistics about the countries in which the cities are located? 

#### WHO Data: Sanitation usage & Health Regulations

In [ ]:
# Merge Numbeo with WHO, merge on country & year
# first add country-period column on numbeo
dfNumbeo['Country_Period'] = (
    dfNumbeo['Country'].str[:10] +
    dfNumbeo['Period'].str[:4]
)
# now add similar column on WHO
dfWHO_sanitation['Country_Period'] = (
    dfWHO_sanitation['name'].str[:10] +
    dfWHO_sanitation['YEAR'].astype(str)
)
dfWHO_SDG['Country_Period'] = (
    dfWHO_SDG['name'].str[:10] +
    dfWHO_SDG['YEAR'].astype(str)
)
# Now merge with sanitation
dfMerged = dfNumbeo.merge(
    dfWHO_sanitation,
    on='Country_Period',
    how='inner'
)
# Renamed column 'Numeric' to 'Sanitation'
dfMerged.rename(columns={'Numeric':'Sanitation'}, inplace=True)
# Now merge with SDG
dfMerged = dfMerged.merge(
    dfWHO_SDG,
    on='Country_Period',
    how='inner'
)
# Renamed column 'Numeric' to 'SDG'
dfMerged.rename(columns={'Numeric':'SDG'}, inplace=True)
# correlation between NumbeoIndex and Sanitation
corr_Numbeo_Sanitation = dfMerged.NumbeoIndex.corr(
    dfMerged.Sanitation, 
    method='spearman')
print(f"Numbeo Index vs Sanitation Usage: {corr_Numbeo_Sanitation}")
# correlation between NumbeoIndex and Health Regulation (SDG)
corr_Numbeo_SDG = dfMerged.NumbeoIndex.corr(
    dfMerged.SDG, 
    method='spearman')
print(f"Numbeo Index vs Health Regulations: {corr_Numbeo_SDG}")

In [ ]:
# fit a linear model
lm = smf.ols(formula='NumbeoIndex ~ Sanitation + SDG', data=dfMerged).fit()
# print the full summary
lm.summary()

With an adjusted $R^2$ value of 0.280 we see that the linear model can already explain
a lot of the variance of the QoL index.

In [ ]:
# add in GDP into dfMerged
gdp = []
for index, row in dfMerged.iterrows():
    country = row['Country']
    year = row['Period'][:4]
    try:
        gdp.append(dfWorldbankGDP.loc[country, year])
    except:
        gdp.append(np.nan)
dfMerged['GDP'] = gdp

In [ ]:
# correlation between NumbeoIndex and GDP
corr_Numbeo_GDP = dfMerged.NumbeoIndex.corr(
    dfMerged.GDP, 
    method='spearman')
print(f"Numbeo Index vs GDP: {corr_Numbeo_GDP}")

In [ ]:
# fit a model
lm = smf.ols(formula='NumbeoIndex ~ Sanitation + GDP', data=dfMerged).fit()
# print the full summary
lm.summary()

Sanitation and GDP can together explain 39% of the variance for the NumbeoIndex.

In [ ]:
# add in emissions damage into dfMerged
emissions = []
for index, row in dfMerged.iterrows():
    country = row['Country']
    year = row['Period'][:4]
    try:
        emissions.append(dfWorldbankEmissions.loc[country, year])
    except:
        emissions.append(np.nan)
dfMerged['Emissions'] = emissions

In [ ]:
# correlation between NumbeoIndex and Emissions
corr_Numbeo_Emissions = dfMerged.NumbeoIndex.corr(
    dfMerged.Emissions, 
    method='spearman')
print(f"Numbeo Index vs Emissions: {corr_Numbeo_Emissions}")

In [ ]:
fig = plt.figure(figsize=(13,13))
g = sns.pairplot(dfMerged.loc[:,['Sanitation','GDP','Emissions','NumbeoIndex']], kind="reg", diag_kind=None)
g.fig.suptitle("Correlation of WHO/WorldBank data and Numbeo Index", y=0.97,fontsize=14)
g.fig.subplots_adjust(top=0.85,left=0.2,bottom=0.1)
g.fig.savefig('visuals/CorrelationWHO_WorldBank_GDP_QoL.png')

In [ ]:
# fit a model
lm = smf.ols(
    formula='NumbeoIndex ~ Sanitation + GDP + Emissions',
    data=dfMerged).fit()
# print the full summary
lm.summary()

Together 43% of the variance of the Numbeo Index can be explained.

## What are the determining characteristics for livability of a city? How do they correlate with cost of living?

In [ ]:
#for this question we will look at the most recent Numbeo data (2020) and make a visual analysis. 
#after that we will train a linear regression model in r and interpret the inference statistics

dfNumbeo2020 = dfNumbeo.set_index('Period').loc['2020',:]
#dfNumbeo2020


#There might be some outliers in the data so we decided to also look at a boxplot:
#fig = plt.figure(figsize=(12,10))
#fig.suptitle('Correlation of City Characteristics', fontsize='x-large', fontweight='bold', y=0.93)
#This is used for the main title. 'figure()' is a class that provides all the plotting elements of a diagram. 
#This must be used first or else the title will not show.fig.subplots_adjust(top=0.85) solves our overlapping title problem.


fig = plt.figure(figsize=(30,15))
#fig.suptitle('Correlation of Rankings in 2020', fontsize='x-large', fontweight='bold')
#fig.subplots_adjust(top=0.88,left=0.2,bottom=0.3)

#ax = fig.add_subplot(111)
g = sns.pairplot(dfNumbeo2020,kind="reg")
g.fig.subplots_adjust(top=0.95)
g.fig.suptitle("Correlation of Rankings with outliers in 2020", y=0.97,fontsize=14)

# plt.title('Correlation of Rankings in 2020', fontdict=fontdict, color="black", position=(0.5,1));

g.fig.savefig('visuals/CorrelationOfRankingsIn2020WithOutliers.png')


In [ ]:
#find the outliers
#taken from https://stackoverflow.com/questions/46827580/multivariate-outlier-removal-with-mahalanobis-distance
def MahalanobisDist(data, verbose=False):
    covariance_matrix = np.cov(data, rowvar=False)
    if is_pos_def(covariance_matrix):
        inv_covariance_matrix = np.linalg.inv(covariance_matrix)
        if is_pos_def(inv_covariance_matrix):
            vars_mean = []
            for i in range(data.shape[0]):
                vars_mean.append(list(data.mean(axis=0)))
            diff = data - vars_mean
            md = []
            for i in range(len(diff)):
                md.append(np.sqrt(diff[i].dot(inv_covariance_matrix).dot(diff[i])))

            if verbose:
                print("Covariance Matrix:\n {}\n".format(covariance_matrix))
                print("Inverse of Covariance Matrix:\n {}\n".format(inv_covariance_matrix))
                print("Variables Mean Vector:\n {}\n".format(vars_mean))
                print("Variables - Variables Mean Vector:\n {}\n".format(diff))
                print("Mahalanobis Distance:\n {}\n".format(md))
            return md
        else:
            print("Error: Inverse of Covariance Matrix is not positive definite!")
    else:
        print("Error: Covariance Matrix is not positive definite!")


def MD_detectOutliers(data, extreme=False, verbose=False):
    MD = MahalanobisDist(data, verbose)
    # one popular way to specify the threshold
    #m = np.mean(MD)
    #t = 3. * m if extreme else 2. * m
    #outliers = []
    #for i in range(len(MD)):
    #    if MD[i] > t:
    #        outliers.append(i)  # index of the outlier
    #return np.array(outliers)

    # or according to the 68–95–99.7 rule
    std = np.std(MD)
    k = 3. * std if extreme else 2. * std
    m = np.mean(MD)
    up_t = m + k
    low_t = m - k
    outliers = []
    for i in range(len(MD)):
        if (MD[i] >= up_t) or (MD[i] <= low_t):
            outliers.append(i)  # index of the outlier
    return np.array(outliers)


def is_pos_def(A):
    if np.allclose(A, A.T):
        try:
            np.linalg.cholesky(A)
            return True
        except np.linalg.LinAlgError:
            return False
    else:
        return False


def handle_outliers(noisy_data):
    """ 
    Parameters
    --------
    noisy_data: data frame that contains outliers
    
    Returns
    --------
    cleaned_data: data frame with outliers
    """
    df = noisy_data 
    #using a multivariate outlier detection based on mahalanobis distance from 
    #https://stackoverflow.com/questions/46827580/multivariate-outlier-removal-with-mahalanobis-distance
    outliers_indices = MD_detectOutliers(df.iloc[:,2:10].values, verbose=False)

    print("Outliers Indices: {}\n".format(outliers_indices))
    df = df.reset_index()
    inliers = pd.Int64Index(np.arange(len(df))).difference(outliers_indices)
    cleaned_data = df.iloc[inliers]
    return cleaned_data
    
dfNumbeo2020_cleaned = handle_outliers(dfNumbeo2020).set_index('Period')


In [ ]:
#looking at the outliers:
dfNumbeo2020.iloc[[0,226],:]

In [ ]:
#plotting again without outliers
#dfNumbeo2020_cleaned

fig = plt.figure(figsize=(30,15))
#fig.suptitle('Correlation of Rankings in 2020', fontsize='x-large', fontweight='bold')

#ax = fig.add_subplot(111)
g = sns.pairplot(dfNumbeo2020_cleaned,kind="reg")
g.fig.suptitle("Correlation of Rankings in 2020", y=0.97,fontsize=14)
g.fig.subplots_adjust(top=0.95)

# plt.title('Correlation of Rankings in 2020', fontdict=fontdict, color="black", position=(0.5,1));

g.fig.savefig('visuals/CorrelationOfRankingsIn2020.png')


In [ ]:
#plotting subset of the features that lead to an increase of the cost of living:
fig = plt.figure(figsize=(30,15))
#fig.suptitle('Correlation of Rankings in 2020', fontsize='x-large', fontweight='bold')

#ax = fig.add_subplot(111)
g = sns.pairplot(dfNumbeo2020_cleaned.loc[:,['CostOfLiving','NumbeoIndex','PurchasingPower','HealthCare']],kind="reg")
g.fig.suptitle("Cost of living vs city characteristics in 2020 - increasing factors", y=0.97,fontsize=14)
g.fig.subplots_adjust(top=0.95)

# plt.title('Correlation of Rankings in 2020', fontdict=fontdict, color="black", position=(0.5,1));

g.fig.savefig('visuals/CostOfLivingCorrelationIn2020.png')


In [ ]:
#plotting subset of the features that lead to a decrease of the cost of living:
fig = plt.figure(figsize=(30,15))
#fig.suptitle('Correlation of Rankings in 2020', fontsize='x-large', fontweight='bold')

#ax = fig.add_subplot(111)
g = sns.pairplot(dfNumbeo2020_cleaned.loc[:,['CostOfLiving','PropertyPriceToIncomeRatio','TrafficCommuteTime','Pollution']],kind="reg")
g.fig.suptitle("Cost of living vs city characteristics in 2020 - decreasing factors", y=0.97,fontsize=14)
g.fig.subplots_adjust(top=0.95)

# plt.title('Correlation of Rankings in 2020', fontdict=fontdict, color="black", position=(0.5,1));

g.fig.savefig('visuals/CostOfLivingCorrelationIn2020NegativeInfluencers.png')


In [ ]:
dfNumbeo2020_cleaned.to_csv('data/NumbeoOutlierRemoved.csv')

In [ ]:
#There might be some outliers in the data so we decided to also look at a boxplot:
fig = plt.figure(figsize=(12,10))
fig.suptitle('Correlation of City Characteristics', fontsize='x-large', fontweight='bold', y=0.93)
#This is used for the main title. 'figure()' is a class that provides all the plotting elements of a diagram. 
#This must be used first or else the title will not show.fig.subplots_adjust(top=0.85) solves our overlapping title problem.
fig.subplots_adjust(top=0.88,left=0.2,bottom=0.3)


ax = fig.add_subplot(111)

fontdict={'fontsize': 14, 'fontweight' : 'book',    'verticalalignment': 'baseline',   'horizontalalignment': 'center'}

#ax.set_title('Plotting Tutorial', fontdict=fontdict)

g = sns.heatmap(dfNumbeo2020_cleaned.corr(),  vmin=-1, cmap='coolwarm', annot=True);

plt.legend(fontsize=14)
#plt.title('Average Rankings over Time', fontdict=fontdict, color="black", position=(0.5,1));
plt.xlabel('', fontsize=14)
plt.ylabel('', fontsize=14);
fig.savefig('visuals/CorrelationOfCityCharacteristics.png')

In [ ]:
#g = sns.boxplot(x="Period", y="value",hue="variable", data=dfNumbeoPivoted,order=np.flip(dfNumbeo.Period.unique()))
#dir(g)

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R -o output
d <- read.csv2("data/NumbeoOutlierRemoved.csv",sep=',',dec ='.')
n <- nrow(d)
set.seed(123)
train <- sample(1:n,round(n*2/3))


res <- lm(CostOfLiving~PurchasingPower+Safety+HealthCare+PropertyPriceToIncomeRatio+TrafficCommuteTime+Pollution+Climate,data=d[train,])
summary(res)
output <- capture.output({summary(res) })


In [ ]:
print(output)

## Interpretation

From the visual analysis we see that there is a strong postive correlation of the cost of living and the quality of life, purchasing power and safety.

The higher the cost of living the lower is the ratio between the property prices and the income. Also the cost of living is lower when the commute time index or the pollution is high.
In addition when the climate is nice the cost of living tends to be higher. This is also confirmed by the inference statistics of the simple regression model. 

The model shows coefficients significant from 0 for all indices. 


And interesting insight is that in order to get good health care you not necessarly have to have high cost of living.


## Are there cities with a higher healthcare ranking and lower cost of living as Vienna?


In [ ]:
ViennaHealthCare = dfNumbeo2020_cleaned.loc[dfNumbeo2020_cleaned.City == 'Vienna','HealthCare'].values[0]
ViennaCostOfLiving = dfNumbeo2020_cleaned.loc[dfNumbeo2020_cleaned.City == 'Vienna','CostOfLiving'].values[0]

dfNumbeo2020_cleaned.loc[((dfNumbeo2020_cleaned.HealthCare >= ViennaHealthCare) & (dfNumbeo2020_cleaned.CostOfLiving <= ViennaCostOfLiving)) ,:]


## Answer
Taipei ranks highest in terms of health care with similar cost of living, but the purchasing power in vienna is higher the pollution is lower and the time to commute is lower well. Thats why vienna scores higher! 